In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/notebook_data/foursquare-location-matching'
%cd
%cd $path

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/root
/content/drive/MyDrive/notebook_data/foursquare-location-matching


In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('train.csv')

In [ ]:
train.head()

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest
0,E_000001272c6c5d,Café Stad Oudenaarde,50.859975,3.634196,Abdijstraat,Nederename,Oost-Vlaanderen,9700,BE,NaN,NaN,Bars,P_677e840bb6fc7e
1,E_000002eae2a589,Carioca Manero,-22.907225,-43.178244,NaN,NaN,NaN,NaN,BR,NaN,NaN,Brazilian Restaurants,P_d82910d8382a83
2,E_000007f24ebc95,ร้านตัดผมการาเกด,13.780813,100.484900,NaN,NaN,NaN,NaN,TH,NaN,NaN,Salons / Barbershops,P_b1066599e78477
3,E_000008a8ba4f48,Turkcell,37.844510,27.844202,Adnan Menderes Bulvarı,NaN,NaN,NaN,TR,NaN,NaN,Mobile Phone Shops,P_b2ed86905a4cd3
4,E_00001d92066153,Restaurante Casa Cofiño,43.338196,-4.326821,NaN,Caviedes,Cantabria,NaN,ES,NaN,NaN,Spanish Restaurants,P_809a884d4407fb


In [ ]:
len(train)

1138812

In [ ]:
null = pd.DataFrame({col: [train[col].isna().sum()/len(train)] for col in train.columns.tolist()})
null.head()

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest
0,0.0,8.781081e-07,0.0,0.0,0.348276,0.26272,0.36932,0.522848,0.00001,0.764909,0.698936,0.086324,0.0


In [ ]:
grouped = train.groupby(['point_of_interest']).size().sort_values(ascending=False)
grouped

point_of_interest
P_fb339198a31db3    332
P_3a3fbd26e16269    199
P_f70a0977377171    166
P_9290e406eb58ac    145
P_fbe61733fcdebe    132
                   ... 
P_6c88990990db38      1
P_6c884940447aa7      1
P_6c8847f66224e2      1
P_6c883c00a983b6      1
P_7ff3b6cf9ac32e      1
Length: 739972, dtype: int64

In [ ]:
#grouped[-500000:-450000]
grouped[: 10]

count    314948.000000
mean          0.056084
std           1.464784
min           0.000000
25%           0.000102
50%           0.000405
75%           0.002309
max         172.792279
Name: longitude, dtype: float64

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
agg = np.zeros(max(grouped) + 1)

for elem in grouped:
  agg[elem] +=1


In [ ]:
print(agg[:6])

[     0. 425024. 277722.  24202.   5937.   2517.]


In [ ]:
poi1 = train[train['point_of_interest'] == 'P_fb339198a31db3'][['categories', 'name',	'latitude',	'longitude']]
print(f'Original length: {len(poi1)}')
print(f'Drop na length: {len(poi1.dropna())}')
poi1

Original length: 332
Drop na length: 121


,categories,name,latitude,longitude
10820,Airport Terminals,Sukarno Hatta airport,-7.329110,108.213608
14686,NaN,Bandara Soekarno Hatta,-6.143365,106.729291
19520,NaN,Bandar Djakarta,-6.117818,106.855710
21192,Airport Terminals,"Bandara Soekarnao Hatta...^^,",-6.234635,106.874453
21433,NaN,Bandara Soekarno Hatta,1.541611,98.915245
...,...,...,...,...
1120868,NaN,Soekarno Hatta International Airport,-7.952919,112.628568
1133655,NaN,Otw Soetta,-6.261004,106.767409
1137194,NaN,Kawasan Soetta,-6.109597,106.653188
1138441,Buildings,Bandara Soekarno Hatta,-6.130148,106.657134


In [ ]:
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from haversine import haversine
a = (	34.486916	,  136.706310)
b = (34.486931, 136.706664)
haversine(a, b)

0.03248805853988205

In [ ]:
# grouped = train.groupby(['point_of_interest']).size().sort_values(ascending=False)
grouped = train.groupby(['point_of_interest'])["latitude"].std().describe()
grouped

count    314948.000000
mean          0.031771
std           0.647925
min           0.000000
25%           0.000084
50%           0.000324
75%           0.001867
max          75.336457
Name: latitude, dtype: float64

In [ ]:
grouped = train.groupby(['point_of_interest'])["longitude"].std().describe()
grouped

count    314948.000000
mean          0.056084
std           1.464784
min           0.000000
25%           0.000102
50%           0.000405
75%           0.002309
max         172.792279
Name: longitude, dtype: float64

In [ ]:
def pariwise_comp(lhs, rhs):
  """
  return True if 2 location is the same POI.
  """
  lhs_pos = (lhs["latitude"], lhs["longitude"])
  rhs_pos = (rhs["latitude"], rhs["longitude"])
  dist = haversine(lhs_pos, rhs_pos)
  if dis < 1:
    return True
  else:
    return False

In [ ]:
data = {'Country':['GB','JP','US'],'Values':[20.2,-10.5,5.7]}
df = pd.DataFrame(data)

In [ ]:
df

,Country,Values
0,GB,20.2
1,JP,-10.5
2,US,5.7


In [ ]:
A=df['Values'].apply(lambda x : df['Values']-x)

In [ ]:
A

,0,1,2
0,0.0,-30.7,-14.5
1,30.7,0.0,16.2
2,14.5,-16.2,0.0
